In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
import cv2
ndvi_list=[]
def save_ndvi_txt(ndvi, path):
    # ensure_dir(path)
    # np.savetxt(path, ndvi, delimiter=',')
    ensure_dir(path)
    with open(path, 'w') as file:
        for row in ndvi:
            line = ' '.join(map(str, row))
            file.write(line + '\n')

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def process_band(image_path, radiance_mult, radiance_add):
    dataset = gdal.Open(image_path)
    band = dataset.GetRasterBand(1)
    data = band.ReadAsArray().astype(np.float32)
    radiance = radiance_mult * data + radiance_add
    reflectance = radiance / np.max(radiance)
    dataset = None
    return reflectance

def compute_ndvi(band4, band3):
    return (band4 - band3) / (band4 + band3)

def find_landsat_images(folder_path):
    landsat_images = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.startswith("LC08"):
                parts = file.split('_')
                date_key = (parts[3], parts[4])  # Using start and end date as key
                if date_key not in landsat_images:
                    landsat_images[date_key] = []
                landsat_images[date_key].append(os.path.join(root, file))
    return landsat_images

def draw_matches(fixed_path, moving_path):
    fixed = cv2.imread(fixed_path, 0)
    moving = cv2.imread(moving_path, 0)
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(fixed, None)
    kp2, des2 = orb.detectAndCompute(moving, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)
    out_img = cv2.drawMatches(fixed, kp1, moving, kp2, matches[:10], None, flags=2)
    return out_img

def save_image(image, path):
  ensure_dir(path)
  plt.imsave(path, image, cmap='gray')

def process_images(images, output_base_path, fixed_image_path, radiance_mult_band_4, radiance_add_band_4, radiance_mult_band_3, radiance_add_band_3):
    for date_key, image_paths in images.items():
        band_4_path = next((p for p in image_paths if 'B4.TIF' in p), None)
        band_3_path = next((p for p in image_paths if 'B3.TIF' in p), None)

        if band_4_path and band_3_path:
            band_4 = process_band(band_4_path, radiance_mult_band_4, radiance_add_band_4)
            band_3 = process_band(band_3_path, radiance_mult_band_3, radiance_add_band_3)
            ndvi = compute_ndvi(band_4, band_3)
            print(f"NDVI for Date: {date_key[0]} - {date_key[1]}")
            print(ndvi)
            # ndvi_list.append(ndvi)

            nir = draw_matches(fixed_image_path, band_4_path)
            red = draw_matches(fixed_image_path, band_3_path)

            output_dir = os.path.join(output_base_path, f"{date_key[0]}_{date_key[1]}")
            ensure_dir(output_dir)

            save_image(nir, os.path.join(output_dir, 'nir_match.png'))
            save_image(red, os.path.join(output_dir, 'red_match.png'))
            # Save NDVI as CSV
            ndvi_txt_path = os.path.join(output_dir, 'ndvi.txt')
            save_ndvi_txt(ndvi, ndvi_txt_path)

# Main
base_folder = '/content/drive/MyDrive/landsat'
output_base_path = '/content/drive/MyDrive/landsat/Output'
fixed_image_path = 'for LC08'

radiance_mult_band_4 = 1.0
radiance_add_band_4 = 10.0
radiance_mult_band_3 = 1.0
radiance_add_band_3 = 10.0

locations = ['test']
for location in locations:
    location_path = os.path.join(base_folder, location)
    images = find_landsat_images(location_path)
    process_images(images, output_base_path, fixed_image_path, radiance_mult_band_4, radiance_add_band_4, radiance_mult_band_3, radiance_add_band_3)


NDVI for Date: 20230828 - 20230828
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
NDVI for Date: 20231031 - 20231031
[[-0.02200488 -0.02200488 -0.02200488 ... -0.02200488 -0.02200488
  -0.02200488]
 [-0.02200488 -0.02200488 -0.02200488 ... -0.02200488 -0.02200488
  -0.02200488]
 [-0.02200488 -0.02200488 -0.02200488 ... -0.02200488 -0.02200488
  -0.02200488]
 ...
 [-0.02200488 -0.02200488 -0.02200488 ... -0.02200488 -0.02200488
  -0.02200488]
 [-0.02200488 -0.02200488 -0.02200488 ... -0.02200488 -0.02200488
  -0.02200488]
 [-0.02200488 -0.02200488 -0.02200488 ... -0.02200488 -0.02200488
  -0.02200488]]
NDVI for Date: 20230913 - 20230913
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
NDVI for Date: 20231015 - 20231015
[[-0.02461745 -0.02461745 -0.02461745 ... -0.02461745 -0

In [ ]:
import os
import numpy as np
import cv2
from osgeo import gdal

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def align_images(im1, im2):
    # Convert images to grayscale (if not already)
    im1_gray = im1 if len(im1.shape) == 2 else cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2_gray = im2 if len(im2.shape) == 2 else cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Initialize ORB detector
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(im1_gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2_gray, None)

    # Create BFMatcher and match the descriptors
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Sort the matches based on distance
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract the matched keypoints
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography to warp image
    height, width = im2.shape[:2]
    im1_aligned = cv2.warpPerspective(im1, h, (width, height))

    return im1_aligned

def process_band(image, radiance_mult, radiance_add):
    # Assuming the image is a numpy array
    radiance = radiance_mult * image + radiance_add
    reflectance = radiance / np.max(radiance)
    return reflectance

def compute_ndvi(band4, band3):
    return (band4 - band3) / (band4 + band3)

def find_landsat_images(folder_path):
    landsat_images = {}
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.startswith("LC09"):
                parts = file.split('_')
                date_key = (parts[3], parts[4])  # Using start and end date as key
                if date_key not in landsat_images:
                    landsat_images[date_key] = []
                landsat_images[date_key].append(os.path.join(root, file))
    return landsat_images

def save_image(image, path):
    ensure_dir(path)
    cv2.imwrite(path, image)

def save_ndvi_txt(ndvi, path):
    # ensure_dir(path)
    # np.savetxt(path, ndvi, delimiter=',')
    ensure_dir(path)
    with open(path, 'w') as file:
        for row in ndvi:
            line = ' '.join(map(str, row))
            file.write(line + '\n')

def process_images(images, output_base_path, fixed_image_path, radiance_mult_band_4, radiance_add_band_4, radiance_mult_band_3, radiance_add_band_3):
    for date_key, image_paths in images.items():
        band_4_path = next((p for p in image_paths if 'B4.TIF' in p), None)
        band_3_path = next((p for p in image_paths if 'B3.TIF' in p), None)

        if band_4_path and band_3_path:
            fixed_image = cv2.imread(fixed_image_path, cv2.IMREAD_GRAYSCALE)
            band_4_image = cv2.imread(band_4_path, cv2.IMREAD_GRAYSCALE)
            band_3_image = cv2.imread(band_3_path, cv2.IMREAD_GRAYSCALE)

            aligned_band_4 = align_images(band_4_image, fixed_image)
            aligned_band_3 = align_images(band_3_image, fixed_image)

            # Process bands
            calibrated_band_4 = process_band(aligned_band_4, radiance_mult_band_4, radiance_add_band_4)
            calibrated_band_3 = process_band(aligned_band_3, radiance_mult_band_3, radiance_add_band_3)

            # Compute NDVI
            ndvi = compute_ndvi(calibrated_band_4, calibrated_band_3)

            output_dir = os.path.join(output_base_path, f"{date_key[0]}_{date_key[1]}")
            ensure_dir(output_dir)

            save_image(aligned_band_4, os.path.join(output_dir, 'aligned_band_4.png'))
            save_image(aligned_band_3, os.path.join(output_dir, 'aligned_band_3.png'))
            save_ndvi_txt(ndvi, os.path.join(output_dir, 'ndvi.txt'))

# Main
base_folder = '/content/drive/MyDrive/landsat'
output_base_path = '/content/drive/MyDrive/landsat/Output'
fixed_image_path = '/content/drive/MyDrive/landsat/test/LC09_L1TP_147049_20230828_20230828_02_T1_B3.TIF'

radiance_mult_band_4 = 1.0
radiance_add_band_4 = 10.0
radiance_mult_band_3 = 1.0
radiance_add_band_3 = 10.0

locations = ['test']
for location in locations:
    location_path = os.path.join(base_folder, location)
    images = find_landsat_images(location_path)
    process_images(images, output_base_path, fixed_image_path, radiance_mult_band_4, radiance_add_band_4, radiance_mult_band_3, radiance_add_band_3)


In [ ]:
import matplotlib.pyplot as plt
import csv
import pandas as pd
import io


df = pd.read_csv('/content/drive/MyDrive/landsat/Output/20230828_20230828/ndvi.csv')
print(df)




      0.000000000000000000e+00  0.000000000000000000e+00.1  \
0                          0.0                         0.0   
1                          0.0                         0.0   
2                          0.0                         0.0   
3                          0.0                         0.0   
4                          0.0                         0.0   
...                        ...                         ...   
7785                       0.0                         0.0   
7786                       0.0                         0.0   
7787                       0.0                         0.0   
7788                       0.0                         0.0   
7789                       0.0                         0.0   

      0.000000000000000000e+00.2  0.000000000000000000e+00.3  \
0                            0.0                         0.0   
1                            0.0                         0.0   
2                            0.0                         0.0   

KeyboardInterrupt: ignored